In [1]:
!pip install transformers
!pip install newsapi-python
!pip install flask-ngrok

     |████████████████████████████████| 552kB 7.5MB/s 
     |████████████████████████████████| 870kB 45.8MB/s 
     |████████████████████████████████| 3.7MB 41.2MB/s 
     |████████████████████████████████| 1.0MB 44.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=ca6e2b1aedf39129d365ebf9c3cb0bd437cc90d4bc5b3ba2c790bcf59173d397
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
from flask import Flask
from flask_ngrok import run_with_ngrok
from newsapi import NewsApiClient
import json
import datetime
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import numpy as np
import requests
from bs4 import BeautifulSoup
import torch

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [0]:
def match_class(target):
    def do_match(tag):
        classes = tag.get('class', [])
        return all(c in classes for c in target)
    return do_match


def get_important_info(articles):
    important = {'articles': []}
    for art in articles['articles']:
        if art['title'] is None or art['description'] is None or \
                art['urlToImage'] is None or art['publishedAt'][:10] is None or \
                art['url'] is None:
            continue
        ness_art = {'title': art['title'],
                    'description': art['description'],
                    'image': art['urlToImage'],
                    'date': art['publishedAt'][:10],
                    'link': art['url']}
        important['articles'].append(ness_art)
    return important


def get_prev_day():
    return str(datetime.date.today()-datetime.timedelta(1))


def generate_text(text, maxlen, model, tokenizer):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    indexed_tokens = tokenizer.encode(text, add_special_tokens=False, return_tensors='pt')
    indexed_tokens = indexed_tokens.to(device)
    output_sequences = model.generate(
        input_ids=indexed_tokens,
        max_length=maxlen,
        temperature=1,
        top_k=0,
        top_p=0.9,
        repetition_penalty=1.0,
        do_sample=True,
        num_return_sequences=1,
    )
    if len(output_sequences.shape) > 2:
        output_sequences.squeeze_()
    generated_sequences = []
    for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
        generated_sequence = generated_sequence.tolist()
        text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)
        total_sequence = (text + text[len(tokenizer.decode(indexed_tokens[0],
                                                           clean_up_tokenization_spaces=True)):])
        generated_sequences.append(total_sequence)
    return clear(generated_sequences[0])


def clear(string):
    cleared = ''
    for i in range(len(string)-1, 0, -1):
        if string[i:] not in string[:i]:
            cleared = string[:i+1]
            break
    return '.'.join(cleared.split('.')[:-1])+'.'

In [5]:
app = Flask(__name__)
run_with_ngrok(app)
newsapi = NewsApiClient(api_key='ac0d27dde82341969f6645b174c34679')


@app.route('/')
def index():
    return "Hello, World!"


def get_top_articles():
    all_articles = newsapi.get_top_headlines(language='en', page=1, page_size=20)
    important = get_important_info(all_articles)
    return json.dumps(important)


@app.route('/getTags', methods=['GET'])
def get_tags():
    url = "https://yandex.ru/news/export"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html')
    classes = soup.find_all(match_class(['export__item']))
    flag = False
    anton = []
    for class_ in classes:
        try:
            tag = class_.contents[0].attrs['href'].split('.rss')[0].split('/')[-1]
            if tag == 'index':
                if not flag:
                    flag = True
                else:
                    break
            anton.append({'title': class_.text, 'tag': tag})
        except:
            break

    return json.dumps(anton, ensure_ascii=False)


@app.route('/getNewsByTag/<string:newsTag>', methods=['GET'])
def get_news_by_tag(newsTag):
    if str(newsTag).lower() == 'top':
        return get_top_articles()
    all_articles = newsapi.get_everything(q=str(newsTag).lower(), language='en', sort_by='relevancy',
                                          page=1, page_size=20, from_param=get_prev_day())
    important = get_important_info(all_articles)
    return json.dumps(important)


@app.route('/genText/<string:prefix>', methods=['GET'])
def gen_text(prefix):
    return json.dumps(generate_text(prefix, np.random.randint(100, 200), model, tokenizer))


if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://dfcdfcfd.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [03/Apr/2020 07:37:59] "GET /getTags HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 07:37:59] "GET /getNewsByTag/Top HTTP/1.1" 200 -
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
127.0.0.1 - - [03/Apr/2020 07:38:23] "GET /genText/Sales%20are%20soaring%20but%20supply%20chain%20disruptions%20mean%20shortages%20loom HTTP/1.1" 200 -
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
127.0.0.1 - - [03/Apr/2020 07:38:42] "GET /genText/The%20ACT%20will%20begin%20actively%20looking%20for%20community%20transmission%20of%20coronavirus%20in%20Canberra. HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 07:39:01] "GET /getNewsByTag/auto HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 07:39:12] "GET /getNewsByTag/world HTTP/1.1" 200 -
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
127.0.0.1 - - [03/Apr/2020 07:39:19] "GET /genText/While%20people%20in%20Europe%20are%20giving%20up%20some%20hard-won%20freedoms%20to%20cont